# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

### Part 1:

For each of the first 5 training examples, print the text of the message along with the label.

In [ ]:
def P1(num_examples=5):
### STUDENT START ###
    for i in range(num_examples):
        print("Training Data: \n %s \n\n and Training label : %s"% (train_data[i], train_labels[i]))  
### STUDENT END ###
P1()

### Part 2:

Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def P2():
    ### STUDENT START ###
    # create the transform
    vectorizer = CountVectorizer()
    # tokenize and build vocab
    vector = vectorizer.fit_transform(dev_data)
    #size of vocabulary
    print('******Size of vocabulary is %i' %len(vectorizer.vocabulary_))
    s=0
    # avg non zero features per example
    v=vector.toarray()
    for i in range (len(v)):
        for j in range(len(v[i])):
            if v[i][j]>0:
                s+=v[i][j]
    avg = s/len(v)
    percent = s/(len(v[1])*len(v))
    print('******Average non-zero features per example are %f' %avg)
    print('******Percent of non-zero features per example are %f' %percent)
    #zero and last feature name sorted alphabetically
    arr=vectorizer.get_feature_names()
    arr.sort()
    print('*******0th and last feature name printed alphabetically**********')
    print(arr[0], arr[-1])
    #build vocabulary
    vectorizer = CountVectorizer(vocabulary = {'atheism','graphics','space','religion'})
    vector = vectorizer.fit_transform(dev_data)
        #size of vocabulary
    print('****After fixed vocabulary of 4 words******')
    print('Shape of training vector', vector.shape)
    v=vector.toarray()
    s=0
    for i in range (len(v)):
            for j in range(len(v[i])):
                if v[i][j]>0:
                    s+=v[i][j]
    avg = s/len(v)

    print('Average non-zero features per example are %f' %avg)
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3))
    vector = vectorizer.fit_transform(dev_data)
    print('*****Vocabulary length using bigram and trigram character features****')
    print(len(vectorizer.vocabulary_))
    vectorizer = CountVectorizer(min_df = 10)
    vector = vectorizer.fit_transform(dev_data)
    print('*****Vocabulary length using min_df****')
    print(len(vectorizer.vocabulary_))
    vectorizer = CountVectorizer()
    vector = vectorizer.fit_transform(train_data)
    print('*****Difference between dev and test data vocab****')
    a = len(vectorizer.vocabulary_)
    vectorizer = CountVectorizer()
    vec = vectorizer.fit_transform(dev_data)
    b = len(vectorizer.vocabulary_)
    print(a-b)
    ### STUDENT END ###
P2()

### Part 3:

Use the default CountVectorizer options and report the f1 score (use metrics.f1_score with average="weighted") for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

* Why doesn't nearest neighbors work well for this problem?
* Any ideas why logistic regression doesn't work as well as Naive Bayes?
* Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [ ]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

def knn(v, dev_labels):
    k = {'n_neighbors': [1,3,5]}
    knn = KNeighborsClassifier()
    grid = GridSearchCV(estimator=knn, param_grid=k)
    grid.fit(v,dev_labels)
    best_k = grid.best_params_
    print('********knn model*********')
    print('Best k parameter is: ', best_k)
    knn_tuned = KNeighborsClassifier(n_neighbors = 5).fit(v,dev_labels)
    predict_labels = knn_tuned.predict(v)
    f1 = f1_score(predict_labels,dev_labels, average="weighted") 
    print('f1 score: %f  '%f1)

def multinom(v, dev_labels):
    alphas = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1]}
    clf = MultinomialNB()
    grid = GridSearchCV(estimator=clf, param_grid=alphas)
    grid.fit(v,dev_labels)
    best_alpha = grid.best_params_
    print('********Multinomial model*********')
    print('Best alpha parameter is: ', best_alpha)
    clf = MultinomialNB(alpha = .001).fit(v,dev_labels)
    predict_labels = clf.predict(v)
    f1 = f1_score(predict_labels,dev_labels, average="weighted") 
    print('f1 score: %f  '%f1)
@ignore_warnings(category=ConvergenceWarning)
def logregr(v, dev_labels):
    c_space = np.logspace(3, 8, 3)
    param_grid = {'C': c_space, 'penalty': ['l2']}
    lr = LogisticRegression()
    grid = GridSearchCV(lr, param_grid, cv = 3)
    grid.fit(v,dev_labels)
    best_C = grid.best_params_
    print('********Logistic regression model*********')
    lr = LogisticRegression(penalty='l2').fit(v, dev_labels)
    predict_labels = lr.predict(v)
    f1 = f1_score(predict_labels,dev_labels, average="weighted") 
    print('Best C parameter is: ', best_C)
    print('f1 score: %f  '%f1)

def P3():
### STUDENT START ###
    vectorizer = CountVectorizer()
    v = vectorizer.fit_transform(dev_data)
    knn(v, dev_labels)
    multinom(v, dev_labels)
    logregr(v, dev_labels)
### STUDENT END ###
P3()

ANSWER: 
Why doesn't nearest neighbors work well for this problem? 
Since our model is based on entire vocabulary and most words are not correlated strongly with output labels, nearest neighbors is not accurate. 

Any ideas why logistic regression doesn't work as well as Naive Bayes?
Logistic regression works better than Naive Bayes in this case.

Logistic regression estimates a weight vector for each class, which you can access with the coef_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.
The sum of squared weights will increase as weights(beta coefficients) increase since l2 regularization parameter penalizes complexity

### Part 4:

Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [ ]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import pandas as pd
@ignore_warnings(category=ConvergenceWarning)
def logregr(v, labels, feat):
    lr = LogisticRegression().fit(v, labels)
    cols = []
    f=[]
    c=[]
    index=[]
    for n in range(4):
        coef = lr.coef_[n].tolist()
        c = (sorted([[x,feat[i],i] for (i,x) in enumerate(coef)], reverse=True )[:5])
        for j in range(len(c)):
            f.append(c[j][1])
            index.append(c[j][2])
        cols.append(n) 
    coef_table = pd.DataFrame(columns = cols, index = f)
    for x in (cols):
        coef = lr.coef_[x].tolist()
        lab=[]
        for i in index:
            lab.append(coef[i]) 
        coef_table[x] = lab

            
    print(coef_table)
def P4():
### STUDENT START ###
    vectorizer = CountVectorizer()
    vector = vectorizer.fit_transform(dev_data)
    v = vector.toarray()
    logregr(v, dev_labels, vectorizer.get_feature_names())
    #Using bigrams
    print('******Using bigrams******')
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
    vector = vectorizer.fit_transform(dev_data)
    v = vector.toarray()
    logregr(v, dev_labels, vectorizer.get_feature_names())
### STUDENT END ###
P4()

ANSWER: There are some interesting features that appear with high weights in both unigram and bigram features. 
Examples are  atheist, an atheist for atheism label
              religion, spam, religion spam for religion label 

In [ ]:
newsgroups_test.target_names

### Part 5:

Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [ ]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from nltk import word_tokenize
def empty_preprocessor(s):
    return s
#trucate each word upto 5 characters
def word_trim(d):
    d=[]
    for i in range(len(dev_data)):
        t=''
        word=word_tokenize(dev_data[i])
        for w in word:
            t= t+ w[:5]+' '
        d.append(t)
    return(d)
def better_preprocessor(l):
### STUDENT START ###
    #lowe case
    l = l.lower()
    #replace all numbers with single token
    l = re.sub('[0-9]+','x',l)
    #removing non letter characters
    l = re.sub('[^a-zA-Z]',' ',l)
    return(l)
### STUDENT END ###
@ignore_warnings(category=ConvergenceWarning)
def P5():
### STUDENT START ###
    #data = better_preprocessor(dev_data)
    vectorizer = CountVectorizer()
    v = vectorizer.fit_transform(dev_data)
    print('***Before pre-processing*****')
    print('Size of dictionary ', len(vectorizer.vocabulary_))
    lr=LogisticRegression()
    pred = lr.fit(v, dev_labels).predict(v)
    print(f1_score(dev_labels, pred, average = "weighted"))
    print('***After pre-processing*****')
    d=word_trim(dev_data)
    vectorizer = CountVectorizer(preprocessor = better_preprocessor)
    v = vectorizer.fit_transform(d)
    print('Size of dictionary ', len(vectorizer.vocabulary_))
    lr=LogisticRegression()
    pred = lr.fit(v, dev_labels).predict(v)
    print(f1_score(dev_labels, pred, average = "weighted")) 
    
### STUDENT END ###
P5()

### Part 6:

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).

In [ ]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
@ignore_warnings(category=ConvergenceWarning)
def P6():
    ### STUDENT START ###
    vectorizer = CountVectorizer()
    v = vectorizer.fit_transform(dev_data)
    print('Original vocabulary count: ', len(vectorizer.vocabulary_))
    vocab_counts=[]
    scores = []
    lr=LogisticRegression(penalty = "l1",solver='liblinear',tol=.015).fit(v,dev_labels)
    vocab=[]
    #This section prints weights for each label that are not zero for L1 regularization
    print('*********L1 regularization***********')
    count=np.zeros(shape = (4,1))
    for i in range(4):
            count[i]=0
            for j in range(len(lr.coef_[i,:])):
                if lr.coef_[i,j]!=0:
                    count[i]+=1 
                    for key, value in vectorizer.vocabulary_.items():
                        if value ==j and key not in vocab:
                            vocab.append(key)
    print('Count of non-zero weights: ', count)
    print('***********L2 regularization**************')
    lr=LogisticRegression(penalty = "l2", tol=.015).fit(v,dev_labels)
    dev=vectorizer.transform(dev_data)
    pred = lr.predict(dev)
    scores.append(accuracy_score(pred,dev_labels))
    vocab_counts.append(len(vectorizer.vocabulary_))
    count=np.zeros(shape = (4,1))
    for i in range(4):
            count[i]=0
            for j in range(len(lr.coef_[i,:])):
                if lr.coef_[i,j]!=0:
                    count[i]+=1 
    print('Count of non-zero weights: ', count)
    print('******Retrained model score*********')
    vectorizer = CountVectorizer(vocabulary = vocab)
    v = vectorizer.fit_transform(dev_data)
    lr=LogisticRegression(penalty = "l2", tol=.015).fit(v,dev_labels)
    dev=vectorizer.transform(dev_data)
    pred = lr.predict(dev)
    scores.append(accuracy_score(pred,dev_labels))
    vocab_counts.append(len(vocab))
    print('Vocabulary size: ', len(vocab))
    print('Prediction score:', accuracy_score(pred,dev_labels))
    plt.scatter(vocab_counts, scores)
    plt.title('Vocabulary counts vs accuracy scores')
    plt.xlabel = 'Vocabulary counts'
    plt.ylabel= 'Accuracy scores'
    plt.plot(vocab_counts, scores, color='red') 
    ### STUDENT END ###

P6()

In [ ]:
ANSWER: With L1 regularization there are far fewer non zero weights compared to L2 regularization where all weights are non zero
        The L1 regularization trimmed the vocabulary by a large number but did not increase the model accuracy

### Part 7:

Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [ ]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import accuracy_score
@ignore_warnings(category=ConvergenceWarning)
def P7():
### STUDENT START ###
#tfidvectorizer 
    vectorizer = TfidfVectorizer()
    v = vectorizer.fit_transform(dev_data, dev_labels)
    test = vectorizer.transform(test_data)
    arr=[]
#training logistic regression model with C=100
    lr = LogisticRegression(penalty='l2', C = 100)
    lr.fit(v, dev_labels)
    pred = lr.predict(test)
#showing ratio R
    for i in range(len(test_labels)):
        ratioR = max(lr.predict_proba(test)[i,:])/lr.predict_proba(test)[i,test_labels[i]]
        arr.append(ratioR)
    x=0
#Printing documents with the three largest values of R
    for x in range(3):
        i = arr.index(max(arr))
        print('\n\n\nDoc: ', dev_data[i])
        print('\nPredicted category: %i' %pred[i])
        print('\nActual category: %i' %test_labels[i])
        print('\n Max 5 feature weights: ')
        feat_weight = v.toarray()[i]*lr.coef_[pred[i]].tolist()
        f=vectorizer.get_feature_names()
        c=(sorted([[x,j,f[j]] for (j,x) in enumerate(feat_weight)], reverse=True)[:5])
        for x in range(len(c)):
            print(c[x][2])
        arr.pop(i)

## STUDENT END ###
P7()

ANSWER: CountVectorizer counts the frequencies of words in corpus.TfidfVectorizer does the same thing but it normalizes the count.
There are a few problems with the wrong predictions above
First, for the three wrong examples, the top five features don't seem to be the correct fit for the actual label.
Second, the model is giving all words equal weight instead of giving more weight to certain important words by labels. For example in the first doc with highest R value, the predicted category is atheism but the actual category is space. This could be because there are more words in the corpus that are related to space. This is evident in the features with the maximum feature weights. 
Third problem could be that the model just uses unigrams. Using bigrams or trigrams may be more effective.